In [202]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
from yellowbrick.classifier import ROCAUC
import klib
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [204]:
cd downloads

[Errno 2] No such file or directory: 'downloads'
/Users/jameslee/Downloads


In [206]:
risk = pd.read_csv('Loan_Default.csv')
#risk = risk.drop(columns=['ID','year','dtir1'])
risk = klib.data_cleaning(risk)

numeric_columns = risk.select_dtypes(exclude=['category', 'int8']).columns.difference(['status'])
risk[numeric_columns] = risk.groupby('status')[numeric_columns].transform(lambda x: x.fillna(x.mean()))

numerical_columns = risk.select_dtypes(exclude=['category','int8']).columns
imputer = SimpleImputer(strategy='median')
risk[numerical_columns] = imputer.fit_transform(risk[numerical_columns])
risk = risk.dropna()
risk = pd.get_dummies(risk, columns=risk.select_dtypes(include=['category']).columns)
risk = klib.data_cleaning(risk)


Shape of cleaned data: (148670, 31) - Remaining NAs: 157014


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 26.65 MB (-75.8%)

Shape of cleaned data: (143983, 69) - Remaining NAs: 0


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 1.43 MB (-6.15%)



In [207]:
risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143983 entries, 0 to 143982
Data columns (total 69 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   loan_amount                         143983 non-null  float32
 1   rate_of_interest                    143983 non-null  float32
 2   interest_rate_spread                143983 non-null  float32
 3   upfront_charges                     143983 non-null  float64
 4   term                                143983 non-null  float32
 5   property_value                      143983 non-null  float32
 6   income                              143983 non-null  float32
 7   credit_score                        143983 non-null  float32
 8   ltv                                 143983 non-null  float32
 9   status                              143983 non-null  int8   
 10  loan_limit_cf                       143983 non-null  boolean
 11  loan_limit_ncf            

In [208]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

numeric_columns = risk.select_dtypes(exclude=['boolean','int8']).columns
risk[numeric_columns] = scaler.fit_transform(risk[numeric_columns])


In [209]:
risk['risk_score'] = risk.sum(axis=1)

In [210]:
risk['risk_score'].head()

0    20.702226
1    20.831288
2    22.966157
3    24.046052
4    23.676927
Name: risk_score, dtype: object

In [211]:
df = pd.read_csv('Loan_Default.csv')
#risk = risk.drop(columns=['ID','year','dtir1'])
df = klib.data_cleaning(df)

numeric_columns = df.select_dtypes(exclude=['category', 'int8']).columns.difference(['status'])
df[numeric_columns] = df.groupby('status')[numeric_columns].transform(lambda x: x.fillna(x.mean()))

numerical_columns = df.select_dtypes(exclude=['category','int8']).columns
imputer = SimpleImputer(strategy='median')
df[numerical_columns] = imputer.fit_transform(df[numerical_columns])
df = df.dropna()

Shape of cleaned data: (148670, 31) - Remaining NAs: 157014


Dropped rows: 0
     of which 0 duplicates. (Rows (first 150 shown): [])

Dropped columns: 0
     of which 0 single valued.     Columns: []
Dropped missing values: 0
Reduced memory by at least: 26.65 MB (-75.8%)



In [212]:
df = df[['credit_score', 'property_value', 'age', 'status', 'loan_amount', 'ltv', 'income']]
df = pd.get_dummies(df, columns=df.select_dtypes(include=['category']).columns)

In [213]:
df['risk_score'] = risk['risk_score']

In [214]:
df.head()

,credit_score,property_value,status,loan_amount,ltv,income,age_25-34,age_35-44,age_45-54,age_55-64,age_65-74,age_<25,age_>74,risk_score
0,758.0,118000.00000,1,116500.0,98.728813,1740.0,True,False,False,False,False,False,False,20.702226
1,552.0,457786.03125,1,206500.0,76.291466,4980.0,False,False,False,True,False,False,False,20.831288
2,834.0,508000.00000,0,406500.0,80.019684,9480.0,False,True,False,False,False,False,False,22.966157
3,587.0,658000.00000,0,456500.0,69.376900,11880.0,False,False,True,False,False,False,False,24.046052
4,602.0,758000.00000,0,696500.0,91.886543,10440.0,True,False,False,False,False,False,False,23.676927


In [215]:
# Consolidate age columns into a single 'age' feature based on the age range categories
import numpy as np

# Define a function to assign age values based on the boolean age range columns
def calculate_age(row):
    if row['age_<25']:
        return 20  # average age for <25 category
    elif row['age_25-34']:
        return 30  # average age for 25-34 category
    elif row['age_35-44']:
        return 40  # average age for 35-44 category
    elif row['age_45-54']:
        return 50  # average age for 45-54 category
    elif row['age_55-64']:
        return 60  # average age for 55-64 category
    elif row['age_65-74']:
        return 70  # average age for 65-74 category
    elif row['age_>74']:
        return 80  # average age for >74 category
    return np.nan  # handle any cases where none apply

# Apply the function to create a single 'age' column
df['age'] = df.apply(calculate_age, axis=1)

# Drop the original age category columns
age_columns = ['age_25-34', 'age_35-44', 'age_45-54', 'age_55-64', 'age_65-74', 'age_<25', 'age_>74']
df = df.drop(columns=age_columns)
df = df.dropna()

In [216]:
features = ['credit_score', 'property_value', 'status', 'loan_amount', 'ltv', 'income', 'age']
X = df[features]
y = df['risk_score']

In [217]:
# Standardize the features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [218]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the cleaned dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Fit a linear regression model to learn the weights
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set and evaluate performance
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
weights = model.coef_

# Display results
print("Mean Squared Error:", mse)
print("Feature Weights:", dict(zip(features, weights)))

Mean Squared Error: 3529.4367352599925
Feature Weights: {'credit_score': -0.05761531634398718, 'property_value': -0.008863050042890508, 'status': -0.3535157132907354, 'loan_amount': -0.24730885462408836, 'ltv': 0.12069516689430103, 'income': -0.023512353192313536, 'age': 0.07827193005652315}


In [234]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

R^2 Score: -0.00023968087652836978
